In [1]:
import numpy as np
import pandas as pd

def get_transition_matrix(val_col):
    transition_matrix = np.zeros((2, 2))

    for i in range(1, len(val_col)):
        today = val_col[i - 1]
        tomorrow = val_col[i]

        if today == 0:
            if tomorrow == 0:
                transition_matrix[0, 0] += 1
            else:
                transition_matrix[0, 1] += 1
        else:
            if tomorrow == 0:
                transition_matrix[1, 0] += 1
            else:
                transition_matrix[1, 1] += 1

    row_sums = transition_matrix.sum(axis=1, keepdims=True)
    transition_matrix = np.divide(
        transition_matrix,
        row_sums,
        out=np.zeros_like(transition_matrix),
        where=row_sums != 0
    )

    return transition_matrix


In [ ]:
matrices = {}

for industry, group in df.groupby('industy'):
    matrices[industry] = get_transition_matrix(group['value'].to_numpy())